# Create data

Creating more useful data from RNA data for training neural nets.

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import torch
from torch import tensor
from torcheval.metrics import MulticlassAccuracy
import torchvision.transforms.functional as TF,torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from miniai.datasets import *
from miniai.conv import *
from miniai.learner import *
from miniai.activations import *
from miniai.init import *
from miniai.sgd import *
from miniai.augment import *
from miniai.xtras import *

In [2]:
from torch.nn import init
from functools import partial
from torch.optim import lr_scheduler
from torch import tensor,nn,optim

In [3]:
import os
path = Path('data')

In [4]:
meta_data = pd.read_csv(path/'meta_data.csv', low_memory=False)
raw_data = pd.read_csv(path/'RNA_count_mat.csv', low_memory=False)
data = raw_data.set_index('gene').T
data = (data - data.mean(axis=0)) / data.std(axis=0)
df = data.join(meta_data.set_index('cell_name'))
df['ZT'] = pd.Categorical(df['ZT'])
df['code'] = df['ZT'].cat.codes
df.head()

,Meg3,Slc1a2,Rian,Cyp2c40,Cyp2c69,Sgcz,Glul,Ugt2b37,Tmeff2,B830012L14Rik,...,Mpp6,Ralgps1,Zfp580,Sept8,Cfap52,Arhgap27,F730043M19Rik,Bora,ZT,code
AAACAGCCAACCCTAA-6,-0.094012,-0.251429,-0.084206,-0.118838,-0.084834,-0.183315,-0.255395,-0.062888,-0.092432,-0.073209,...,1.122616,2.084872,-0.036734,-0.036734,-0.036734,-0.036734,-0.036734,-0.318222,ZT18,4
AAACCAACAGTAAAGC-6,-0.094012,-0.032904,-0.084206,-0.118838,-0.084834,-0.183315,-0.255395,-0.062888,-0.092432,-0.073209,...,-0.554376,-0.619790,-0.036734,-0.036734,-0.036734,-0.036734,-0.036734,2.058709,ZT18,4
AAACCGAAGGTCCTAG-6,-0.094012,-0.251429,-0.084206,-0.118838,-0.084834,-0.183315,-0.255395,-0.062888,-0.092432,-0.073209,...,0.451819,0.281764,-0.036734,-0.036734,-0.036734,-0.036734,-0.036734,-0.318222,ZT18,4
AAAGCACCATTGTCCT-6,-0.094012,-0.251429,-0.084206,-0.118838,-0.084834,-0.183315,-0.255395,-0.062888,-0.092432,-0.073209,...,0.451819,0.281764,-0.036734,-0.036734,-0.036734,-0.036734,-0.036734,-0.318222,ZT18,4
AAAGGACGTGAGGTGA-6,-0.094012,-0.251429,-0.084206,-0.118838,-0.084834,-0.183315,-0.255395,-0.062888,-0.092432,-0.073209,...,-0.889775,1.183318,-0.036734,-0.036734,-0.036734,-0.036734,-0.036734,-0.318222,ZT18,4


In [15]:
meta_data.head()

,cell_name,ZT
0,AAACAGCCAACCCTAA-6,ZT18
1,AAACCAACAGTAAAGC-6,ZT18
2,AAACCGAAGGTCCTAG-6,ZT18
3,AAAGCACCATTGTCCT-6,ZT18
4,AAAGGACGTGAGGTGA-6,ZT18


In [8]:
df.to_csv(path/'RNA_norm.csv', index_label='cell')

In [9]:
pd.read_csv(path/'RNA_norm.csv', index_col='cell', low_memory=False).head()

,Meg3,Slc1a2,Rian,Cyp2c40,Cyp2c69,Sgcz,Glul,Ugt2b37,Tmeff2,B830012L14Rik,...,Mpp6,Ralgps1,Zfp580,Sept8,Cfap52,Arhgap27,F730043M19Rik,Bora,ZT,code
cell,,,,,,,,,,,,,,,,,,,,,
AAACAGCCAACCCTAA-6,-0.094012,-0.251429,-0.084206,-0.118838,-0.084834,-0.183315,-0.255395,-0.062888,-0.092432,-0.073209,...,1.122616,2.084872,-0.036734,-0.036734,-0.036734,-0.036734,-0.036734,-0.318222,ZT18,4
AAACCAACAGTAAAGC-6,-0.094012,-0.032904,-0.084206,-0.118838,-0.084834,-0.183315,-0.255395,-0.062888,-0.092432,-0.073209,...,-0.554376,-0.619790,-0.036734,-0.036734,-0.036734,-0.036734,-0.036734,2.058709,ZT18,4
AAACCGAAGGTCCTAG-6,-0.094012,-0.251429,-0.084206,-0.118838,-0.084834,-0.183315,-0.255395,-0.062888,-0.092432,-0.073209,...,0.451819,0.281764,-0.036734,-0.036734,-0.036734,-0.036734,-0.036734,-0.318222,ZT18,4
AAAGCACCATTGTCCT-6,-0.094012,-0.251429,-0.084206,-0.118838,-0.084834,-0.183315,-0.255395,-0.062888,-0.092432,-0.073209,...,0.451819,0.281764,-0.036734,-0.036734,-0.036734,-0.036734,-0.036734,-0.318222,ZT18,4
AAAGGACGTGAGGTGA-6,-0.094012,-0.251429,-0.084206,-0.118838,-0.084834,-0.183315,-0.255395,-0.062888,-0.092432,-0.073209,...,-0.889775,1.183318,-0.036734,-0.036734,-0.036734,-0.036734,-0.036734,-0.318222,ZT18,4


In [11]:
raw_bulk_data = pd.read_csv(path/'RNA.csv', low_memory=False)
bulk_data = raw_bulk_data.set_index('ID').T
bulk_data = (bulk_data - bulk_data.mean(axis=0)) / bulk_data.std(axis=0)
bulk_data['code'] = [0,1,2,3,4,5,0,1,2,3,4,5]

In [12]:
df2 = pd.DataFrame(np.random.normal(size=(15992, 9482)), columns=bulk_data.columns)
df.reset_index(inplace=True)
df2.update(df)

In [16]:
df2['cell'] = meta_data['cell_name']

In [23]:
df2 = df2.set_index('cell')

In [24]:
df2.head()

ID,2810459M11Rik,Abcb11,Acot7,Ahctf1,Ap2a2,Asb13,Asl,Chka,Clic3,Clpx,...,Ap1s3,Vamp4,Pigm,Pard6g,Gas2,Pot1b,Zfp759,Neo1,Sco1,code
cell,,,,,,,,,,,,,,,,,,,,,
AAACAGCCAACCCTAA-6,-0.540902,1.084080,0.463838,0.436680,-0.351168,-0.658594,-0.043890,1.686979,-1.087222,0.234600,...,-0.166754,-0.719685,0.263727,-0.245295,2.159961,-0.401256,0.501759,-1.400617,0.561399,4.0
AAACCAACAGTAAAGC-6,-0.452915,-0.872108,0.040194,0.738373,1.136242,-0.126162,-0.387448,-0.189915,-1.491271,-0.363165,...,-0.166754,1.487384,-0.839048,-0.615568,-0.784713,-0.401256,1.202853,-0.226444,-1.835237,4.0
AAACCGAAGGTCCTAG-6,1.085506,0.738870,-0.336613,-0.546819,-1.556767,1.386236,1.673901,0.122900,-0.548618,1.131247,...,10.777388,-0.075675,0.247874,0.280195,0.821473,-0.401256,0.940912,-0.728631,0.228285,4.0
AAAGCACCATTGTCCT-6,-0.346411,0.681335,0.677034,0.980449,-0.632364,1.077614,0.643226,-0.033508,-0.725366,1.729012,...,-0.166754,0.056551,-1.058772,-0.208770,0.821473,-0.401256,-1.485775,-0.475149,1.874629,4.0
AAAGGACGTGAGGTGA-6,0.557443,-0.641969,-0.087715,0.643695,-0.266904,0.422090,-0.387448,0.435716,0.828472,-0.512606,...,-0.166754,-1.023835,1.493227,2.312298,-0.784713,-0.401256,1.787818,-1.085683,-0.709830,4.0


In [26]:
df2.to_csv(path/'RNA_batched.csv', index_label='cell')
pd.read_csv(path/'RNA_batched.csv', index_col='cell', low_memory=False).head()

,2810459M11Rik,Abcb11,Acot7,Ahctf1,Ap2a2,Asb13,Asl,Chka,Clic3,Clpx,...,Ap1s3,Vamp4,Pigm,Pard6g,Gas2,Pot1b,Zfp759,Neo1,Sco1,code
cell,,,,,,,,,,,,,,,,,,,,,
AAACAGCCAACCCTAA-6,-0.540902,1.084080,0.463838,0.436680,-0.351168,-0.658594,-0.043890,1.686979,-1.087222,0.234600,...,-0.166754,-0.719685,0.263727,-0.245295,2.159961,-0.401256,0.501759,-1.400617,0.561399,4.0
AAACCAACAGTAAAGC-6,-0.452915,-0.872108,0.040194,0.738373,1.136242,-0.126162,-0.387448,-0.189915,-1.491271,-0.363165,...,-0.166754,1.487384,-0.839048,-0.615568,-0.784713,-0.401256,1.202853,-0.226444,-1.835237,4.0
AAACCGAAGGTCCTAG-6,1.085506,0.738870,-0.336613,-0.546819,-1.556767,1.386236,1.673901,0.122900,-0.548618,1.131247,...,10.777388,-0.075675,0.247874,0.280195,0.821473,-0.401256,0.940912,-0.728631,0.228285,4.0
AAAGCACCATTGTCCT-6,-0.346411,0.681335,0.677034,0.980449,-0.632364,1.077614,0.643226,-0.033508,-0.725366,1.729012,...,-0.166754,0.056551,-1.058772,-0.208770,0.821473,-0.401256,-1.485775,-0.475149,1.874629,4.0
AAAGGACGTGAGGTGA-6,0.557443,-0.641969,-0.087715,0.643695,-0.266904,0.422090,-0.387448,0.435716,0.828472,-0.512606,...,-0.166754,-1.023835,1.493227,2.312298,-0.784713,-0.401256,1.787818,-1.085683,-0.709830,4.0
